In [109]:
#imports and scraping raw data from site
#df is the actual betting line
#df2 has other info, mostly using to get player id to name dict

import pandas as pd
import datetime as dt
import re
from unidecode import unidecode
from betting_functions import get_url_json

In [110]:
dog_props_url = 'https://api.underdogfantasy.com/beta/v3/over_under_lines'
data = get_url_json(dog_props_url)

In [111]:
data.keys()

dict_keys(['appearances', 'games', 'over_under_lines', 'players', 'solo_games'])

In [112]:
df = pd.json_normalize(list(data.values()))
df.index = data.keys()
new = df.T['over_under_lines'].values
lines = pd.json_normalize(new)
lines.sample(2)

,id,live_event,live_event_stat,options,over_under_id,rank,stat_value,status,over_under.id,over_under.appearance_stat.id,over_under.appearance_stat.appearance_id,over_under.appearance_stat.display_stat,over_under.appearance_stat.graded_by,over_under.appearance_stat.pickem_stat_id,over_under.appearance_stat.stat,over_under.boost,over_under.scoring_type_id,over_under.title
1410,bf6ca083-8a0b-4de0-948a-f9d60d2585bb,False,None,[{'id': '9a11fc49-509e-4e95-84aa-ab6b23a2f030'...,a2158c74-52d5-441f-8df1-7c963ffb8989,25186169,15.5,active,a2158c74-52d5-441f-8df1-7c963ffb8989,98cb159e-0101-4813-845a-684e99a8a9e0,73b3330f-e443-40c6-87b4-25b29ace36de,Headshots on Maps 1+2,high_score,fe03999c-170d-4ad5-a30c-6ea4eb44bfbd,headshots_on_maps_1_2,None,None,CS: hybrid Headshots on Maps 1+2 O/U
1293,f2c5fc04-d141-4169-a1b3-1b67262aaf08,False,None,[{'id': '9392e0a3-6296-4236-bee9-2ad5fde4b60b'...,77d87d65-6ec4-474c-8647-dfca81100cb2,25180061,28.0,active,77d87d65-6ec4-474c-8647-dfca81100cb2,a3e241df-bdf9-4ab6-9e7c-44902a70fda0,1acfbd6a-7486-42a2-a10e-ef952097422f,Kills on Map 1+2,high_score,fd4d222d-d4d0-4258-b0d2-d225e1947b0c,map_1_and_2_kills,None,None,CS: br0 Kills on Map 1+2 O/U


In [113]:
test =  pd.json_normalize(df.T['players'])
player_df = test.copy(deep=True)
player_df = player_df[['id','first_name', 'last_name',
       'sport_id','team_id']]
player_df = player_df.dropna()#

player_df.loc[player_df.sport_id=='ESPORTS']
x = player_df[player_df.sport_id=='ESPORTS']
p2esport = x[['last_name','first_name']].to_dict('split')
p2esport = dict(p2esport['data'])

In [114]:
test =  pd.json_normalize(df.T['players'])
player_df = test.copy(deep=True)
player_df = player_df[['id','first_name', 'last_name',
       'sport_id','team_id']]
player_df = player_df.dropna()
player_df.loc[player_df['sport_id'] != 'ESPORTS', 'full_name'] = player_df['first_name']+' ' + player_df['last_name']
player_df.loc[player_df['sport_id'] == 'ESPORTS', 'full_name'] = player_df['last_name']
#player_df['full_name'] = player_df['first_name']+' ' + player_df['last_name']
player_df = player_df[['full_name','sport_id','team_id']]
p2l = player_df[['full_name','sport_id']].to_dict('split')
p2tm = player_df[['full_name','team_id']].to_dict('split')
temp = p2l['data']
temp2 = p2tm['data']
p2l = dict(temp)
p2tm = dict(temp2)

In [115]:
player_df

,full_name,sport_id,team_id
0,Jalen Brunson,NBA,c71a1a59-28ee-4ac6-ba44-d8aeb0d2ac3e
1,Julius Randle,NBA,c71a1a59-28ee-4ac6-ba44-d8aeb0d2ac3e
2,RJ Barrett,NBA,c71a1a59-28ee-4ac6-ba44-d8aeb0d2ac3e
3,Mitchell Robinson,NBA,c71a1a59-28ee-4ac6-ba44-d8aeb0d2ac3e
4,Quentin Grimes,NBA,c71a1a59-28ee-4ac6-ba44-d8aeb0d2ac3e
...,...,...,...
471,C. Seager + M. Semien,MLBCOMBOS,976becd7-ee88-4db5-9ae6-d1bb6b2b16a8
472,A. Garcia + N. Lowe,MLBCOMBOS,976becd7-ee88-4db5-9ae6-d1bb6b2b16a8
473,J. Jung + J. Heim,MLBCOMBOS,976becd7-ee88-4db5-9ae6-d1bb6b2b16a8
474,C. Walker + T. Pham,MLBCOMBOS,db69f844-d6eb-4332-b133-796ce316890a


In [116]:
player_df.sport_id.unique()

array(['NBA', 'MLB', 'NHL', 'NFL', 'CFB', 'SOCCER', 'ESPORTS',
       'BASKETBALL', 'MLBCOMBOS'], dtype=object)

In [117]:
first2 = player_df['full_name'].apply(lambda x: ' '.join(x.split()[0:2]))
names = pd.DataFrame((first2,player_df.full_name))
names = names.T
names.columns=['first2','fullname']
names.loc[names.first2=='Jazz Chisholm']
first2full = names.to_dict('split')
first2full = dict(first2full['data'])

In [118]:
games = pd.json_normalize(df.T['games'].values).dropna(how='all')
#games = games.loc[games['sport_id']=='NBA']
games.tail(3)

,id,away_team_id,away_team_score,home_team_id,home_team_score,match_progress,period,scheduled_at,season_type,sport_id,status,title,type,year
48,57260.0,2625abaf-caad-4404-8647-1593aef46853,0.0,597df82f-6232-44ce-98ec-e1a6f885eff4,0.0,Wed 06:00am,0.0,2023-11-01T10:00:00Z,regular,ESPORTS,scheduled,CDF: Team Du Sud vs. Team GO,Game,2023.0
49,57299.0,b718ef39-628e-4a03-b075-bce5701748ee,0.0,5688f051-a6cb-420d-82dd-cce79e3517d6,0.0,Wed 12:00pm,0.0,2023-11-01T16:00:00Z,regular,ESPORTS,scheduled,Val: Giants Gaming vs. SAW,Game,2023.0
50,57297.0,359b27c1-5804-4959-9eb5-6eee2cc6795f,0.0,73019268-6016-4a30-8734-24935592408c,0.0,Tue 03:30pm,0.0,2023-10-31T19:30:00Z,regular,BASKETBALL,scheduled,Maccabi Tel-Aviv @ Milano,Game,2023.0


In [119]:
def to_central(date_str):
    x = pd.to_datetime(date_str)
    x = x.tz_convert('US/Central')
    return str(x.date())
to_central('2023-05-20T00:40:00Z')

'2023-05-19'

In [120]:
games.scheduled_at = games.scheduled_at.apply(to_central)

In [121]:
#grabbing team names from match title
matchup = games['title']
home = []
away = []
for i in matchup:
    temp = re.split(' @ |, |vs.',i)
    home.append(temp[0])
    away.append(temp[1])
#fixing esports team names
for i in range(len(home)):
    home[i] = home[i].split(': ')[-1]
    
for i in range(len(away)):
    away[i] = away[i].split(': ')[-1]

matchup = pd.DataFrame((away,home,games.away_team_id,games.home_team_id)).T
matchup.columns = ['Home', 'Away','away_id','home_id']
dates = games['scheduled_at'].values
#temp = []
#for i in dates:
#    temp.append(i.split('T')[0])
matchup['Date'] = dates
id2tm =  matchup[['home_id','Home']].to_dict('split')
id2tmaway =  matchup[['away_id','Away']].to_dict('split')
id2tm = dict(id2tm['data'])
id2tmaway = dict(id2tmaway['data'])
id2tm.update(id2tmaway)

In [122]:
games

,id,away_team_id,away_team_score,home_team_id,home_team_score,match_progress,period,scheduled_at,season_type,sport_id,status,title,type,year
0,52407.0,c71a1a59-28ee-4ac6-ba44-d8aeb0d2ac3e,0.0,56fcd081-3e21-406e-af95-5716abbfba2e,0.0,Tue 07:30pm,0.0,2023-10-31,regular,NBA,scheduled,NYK @ CLE,Game,2023.0
1,52408.0,fee4a894-1970-402a-873b-8faea88f4ec9,0.0,f98f3baa-ad03-4188-9e8a-0cb52e1b900e,0.0,Tue 10:00pm,0.0,2023-10-31,regular,NBA,scheduled,SAS @ PHX,Game,2023.0
2,52409.0,27c02dfc-343a-432b-8a64-c963035b609d,0.0,bd4b60cd-5adb-4cc6-8e8a-97084501f386,0.0,Tue 10:30pm,0.0,2023-10-31,regular,NBA,scheduled,ORL @ LAC,Game,2023.0
3,56920.0,f85132d6-ce4f-461f-ab0d-97810d017c94,0.0,0331354e-f6cb-4dba-ba55-7c5c40ab9f5e,0.0,Tue 08:03pm,0.0,2023-10-31,post,MLB,scheduled,TEX @ ARI,Game,2023.0
4,50480.0,2904aaa7-ffcf-4717-a723-28521efcc8e0,0.0,dc5264a9-6ed2-4a86-aeac-a8051c2e6c6d,0.0,Tue 08:00pm,0.0,2023-10-31,regular,NHL,scheduled,LA @ TOR,Game,2023.0
5,50481.0,e6fa00a2-55eb-45f3-b8de-2a7ad7329f30,0.0,fb8cd80e-6c4d-4935-ada0-2688ed01e203,0.0,Tue 10:00pm,0.0,2023-10-31,regular,NHL,scheduled,NSH @ VAN,Game,2023.0
6,39429.0,f96aa8db-21c2-5b86-b49d-7e64b4eda61d,0.0,ecc8eb1b-f714-57a6-bcf3-b183dd6c12a8,0.0,Thu 08:15pm,0.0,2023-11-02,regular,NFL,scheduled,TEN @ PIT,Game,2023.0
7,39430.0,9153e0a9-da83-5246-ba43-417537f1bcce,0.0,a6f458f4-5078-56e4-a839-af96f1191314,0.0,Sun 09:30am,0.0,2023-11-05,regular,NFL,scheduled,MIA @ KC,Game,2023.0
8,39432.0,31631011-5902-52f6-ba01-c4c8d8eb3fd9,0.0,01bfe9d5-f671-57ed-aa60-249fcca9267c,0.0,Sun 01:00pm,0.0,2023-11-05,regular,NFL,scheduled,SEA @ BAL,Game,2023.0
9,39437.0,d150534e-6a05-587b-b9e3-50ef86602e20,0.0,a2fadaac-562f-5a7e-bd63-6d88d90c1ac4,0.0,Sun 01:00pm,0.0,2023-11-05,regular,NFL,scheduled,LAR @ GB,Game,2023.0


In [123]:
id2tm

{'56fcd081-3e21-406e-af95-5716abbfba2e': 'CLE',
 'f98f3baa-ad03-4188-9e8a-0cb52e1b900e': 'PHX',
 'bd4b60cd-5adb-4cc6-8e8a-97084501f386': 'LAC',
 '0331354e-f6cb-4dba-ba55-7c5c40ab9f5e': 'ARI',
 'dc5264a9-6ed2-4a86-aeac-a8051c2e6c6d': 'TOR',
 'fb8cd80e-6c4d-4935-ada0-2688ed01e203': 'VAN',
 'ecc8eb1b-f714-57a6-bcf3-b183dd6c12a8': 'PIT',
 'a6f458f4-5078-56e4-a839-af96f1191314': 'KC',
 '01bfe9d5-f671-57ed-aa60-249fcca9267c': 'BAL',
 'a2fadaac-562f-5a7e-bd63-6d88d90c1ac4': 'GB',
 'ef876de9-81ac-5e60-af6e-0ca1700f3a51': 'NE',
 '4ab08caa-b79d-598e-82ef-0906a60d2a89': 'HOU',
 '530518ed-91db-57c4-9077-27cc0dd9a293': 'NO',
 '8699a914-7d44-5a36-b7a6-2063d3ea761f': 'ATL',
 '17f3bc4a-e2d6-5dc5-a554-00549ff0139f': 'CAR',
 'de7219e5-92f1-5989-804a-68479055ba42': 'PHI',
 '5ce78c37-b02c-52da-9e4f-fcd65b6ccb76': 'LV',
 'ab235cf0-a041-5d36-8241-90a90f0dcb5e': 'CIN',
 '34d3cafb-c2bf-40a3-8a2d-9af0785d7494': 'CMC',
 '84926908-f59f-4507-86ab-c3708e1e1bdf': 'TOL',
 'd8c72ef1-b248-4e6a-84d0-beb9b5f99392': 'TT'

In [124]:
lines = lines[['stat_value','over_under.appearance_stat.stat', 'over_under.title']]
lines
players = lines['over_under.title'].values
guys = []
for i in players:
    temp = i.split()
    final = temp[0] + ' '+temp[1]
    guys.append(final)
    
guys
lines= lines.rename(columns={'stat_value':'Line','over_under.appearance_stat.stat':'Stat', 'over_under.title':'title'})
lines['Player']= guys
lines['Player'] = lines['Player'].apply(lambda x: unidecode(x).replace('_',' '))


In [125]:
def map_name(x):
    if x is None:
        return None
    if x in first2full:
        return first2full[x]
    elif x.split(': ')[-1] in first2full:
        return x.split(': ')[-1]
    elif x.split()[-1] in first2full:
        return x.split()[-1]
    else:
        return x

lines['Player'] = lines['Player'].apply(lambda x: map_name(x))
lines.Player.unique()

array(['Jalen Brunson', 'Julius Randle', 'RJ Barrett',
       'Mitchell Robinson', 'Quentin Grimes', 'Donovan Mitchell',
       'Evan Mobley', 'Victor Wembanyama', 'Keldon Johnson',
       'Devin Vassell', 'Jeremy Sochan', 'Zach Collins', 'Kevin Durant',
       'Jusuf Nurkic', 'Josh Okogie', 'Grayson Allen', 'Eric Gordon',
       'Paolo Banchero', 'Franz Wagner', 'Wendell Carter Jr.',
       'Markelle Fultz', 'Jalen Suggs', 'Kawhi Leonard', 'Paul George',
       'Russell Westbrook', 'Ivica Zubac', 'Andrew Heaney',
       'Corey Seager', 'Marcus Semien', 'Adolis Garcia', 'Josh Jung',
       'Nathaniel Lowe', 'Mitch Garver', 'Jonah Heim', 'Leody Taveras',
       'Evan Carter', 'Corbin Carroll', 'Ketel Marte', 'Christian Walker',
       'Tommy Pham', 'Gabriel Moreno', 'Evan Longoria', 'Geraldo Perdomo',
       'Lourdes Gurriel Jr.', 'Cam Talbot', 'Kevin Fiala', 'Adrian Kempe',
       'Pierre-Luc Dubois', 'Anze Kopitar', 'Phillip Danault',
       'Drew Doughty', 'Trevor Moore', 'Matt Roy',

In [126]:
league = []
for p in guys:
    if p in p2l:
        league.append(p2l[p])
    elif p.split()[-1] in p2l:
        league.append(p2l[p.split()[-1]])
    else:
        league.append('other')
league[0:5]

['NBA', 'NBA', 'NBA', 'NBA', 'NBA']

In [127]:
guys[1000:1007]

['LoL: Kiin',
 'LoL: Kiin',
 'LoL: Kiin',
 'LoL: Kiin',
 'LoL: Kiin',
 'LoL: Cuzz',
 'LoL: Cuzz']

In [128]:
team_id = []
for p in guys:
    if p in p2l:
        team_id.append(p2tm[p])
    elif p.split()[-1] in p2l:
        team_id.append(p2tm[p.split()[-1]])
    else:
        team_id.append('other')
team_id[0:5]

['c71a1a59-28ee-4ac6-ba44-d8aeb0d2ac3e',
 'c71a1a59-28ee-4ac6-ba44-d8aeb0d2ac3e',
 'c71a1a59-28ee-4ac6-ba44-d8aeb0d2ac3e',
 'c71a1a59-28ee-4ac6-ba44-d8aeb0d2ac3e',
 'c71a1a59-28ee-4ac6-ba44-d8aeb0d2ac3e']

In [129]:
lines['League'] = league
lines

,Line,Stat,title,Player,League
0,22.5,points,Jalen Brunson Points O/U,Jalen Brunson,NBA
1,30.5,pts_rebs_asts,Jalen Brunson Pts + Rebs + Asts O/U,Jalen Brunson,NBA
2,5.0,assists,Jalen Brunson Assists O/U,Jalen Brunson,NBA
3,8.5,rebs_asts,Jalen Brunson Rebounds + Assists O/U,Jalen Brunson,NBA
4,25.5,pts_rebs,Jalen Brunson Points + Rebounds O/U,Jalen Brunson,NBA
...,...,...,...,...,...
1546,16.5,fantasy_points,C. Seager + M. Semien Fantasy Points O/U,C. Seager + M. Semien,other
1547,16.5,fantasy_points,A. Garcia + N. Lowe Fantasy Points O/U,A. Garcia + N. Lowe,other
1548,13.5,fantasy_points,J. Jung + J. Heim Fantasy Points O/U,J. Jung + J. Heim,other
1549,14.5,fantasy_points,C. Walker + T. Pham Fantasy Points O/U,C. Walker + T. Pham,other


In [130]:
def map_team(x):
    if x in p2tm:
        return p2tm[x]
    elif p.split()[-1] in p2l:
        return p2tm[p.split()[-1]]
    else:
        return x

lines['team_id'] = lines.Player.apply(lambda x: map_team(x))
lines

,Line,Stat,title,Player,League,team_id
0,22.5,points,Jalen Brunson Points O/U,Jalen Brunson,NBA,c71a1a59-28ee-4ac6-ba44-d8aeb0d2ac3e
1,30.5,pts_rebs_asts,Jalen Brunson Pts + Rebs + Asts O/U,Jalen Brunson,NBA,c71a1a59-28ee-4ac6-ba44-d8aeb0d2ac3e
2,5.0,assists,Jalen Brunson Assists O/U,Jalen Brunson,NBA,c71a1a59-28ee-4ac6-ba44-d8aeb0d2ac3e
3,8.5,rebs_asts,Jalen Brunson Rebounds + Assists O/U,Jalen Brunson,NBA,c71a1a59-28ee-4ac6-ba44-d8aeb0d2ac3e
4,25.5,pts_rebs,Jalen Brunson Points + Rebounds O/U,Jalen Brunson,NBA,c71a1a59-28ee-4ac6-ba44-d8aeb0d2ac3e
...,...,...,...,...,...,...
1546,16.5,fantasy_points,C. Seager + M. Semien Fantasy Points O/U,C. Seager + M. Semien,other,976becd7-ee88-4db5-9ae6-d1bb6b2b16a8
1547,16.5,fantasy_points,A. Garcia + N. Lowe Fantasy Points O/U,A. Garcia + N. Lowe,other,976becd7-ee88-4db5-9ae6-d1bb6b2b16a8
1548,13.5,fantasy_points,J. Jung + J. Heim Fantasy Points O/U,J. Jung + J. Heim,other,976becd7-ee88-4db5-9ae6-d1bb6b2b16a8
1549,14.5,fantasy_points,C. Walker + T. Pham Fantasy Points O/U,C. Walker + T. Pham,other,db69f844-d6eb-4332-b133-796ce316890a


In [131]:
def map_id(x):
    if x in id2tm:
        return id2tm[x]
    else:
        return x
#lines['team_id'] = lines.team_id.apply(lambda x: map_id(x))


In [132]:
final = pd.merge(lines,matchup, how='left',left_on='team_id',right_on='away_id')
final = final.drop(['Home', 'Away','home_id','away_id'], axis=1)
final2 = pd.merge(final,matchup, how='left',left_on='team_id',right_on='home_id')
final2 = final2.drop(['Home', 'Away','home_id','away_id'], axis=1)
final2['Date'] = final2['Date_x'].fillna(final2['Date_y'])
final2 = final2.drop(['Date_x', 'Date_y'], axis=1)
final2['team_id'] = final2.team_id.apply(lambda x: map_id(x))
final2#.loc[final.Player=='Jimmy Butler']

,Line,Stat,title,Player,League,team_id,Date
0,22.5,points,Jalen Brunson Points O/U,Jalen Brunson,NBA,NYK,2023-10-31
1,30.5,pts_rebs_asts,Jalen Brunson Pts + Rebs + Asts O/U,Jalen Brunson,NBA,NYK,2023-10-31
2,5.0,assists,Jalen Brunson Assists O/U,Jalen Brunson,NBA,NYK,2023-10-31
3,8.5,rebs_asts,Jalen Brunson Rebounds + Assists O/U,Jalen Brunson,NBA,NYK,2023-10-31
4,25.5,pts_rebs,Jalen Brunson Points + Rebounds O/U,Jalen Brunson,NBA,NYK,2023-10-31
...,...,...,...,...,...,...,...
1566,16.5,fantasy_points,C. Seager + M. Semien Fantasy Points O/U,C. Seager + M. Semien,other,976becd7-ee88-4db5-9ae6-d1bb6b2b16a8,NaN
1567,16.5,fantasy_points,A. Garcia + N. Lowe Fantasy Points O/U,A. Garcia + N. Lowe,other,976becd7-ee88-4db5-9ae6-d1bb6b2b16a8,NaN
1568,13.5,fantasy_points,J. Jung + J. Heim Fantasy Points O/U,J. Jung + J. Heim,other,976becd7-ee88-4db5-9ae6-d1bb6b2b16a8,NaN
1569,14.5,fantasy_points,C. Walker + T. Pham Fantasy Points O/U,C. Walker + T. Pham,other,db69f844-d6eb-4332-b133-796ce316890a,NaN


In [133]:
final2.columns = ['Line', 'Stat', 'title', 'Player', 'League', 'Team', 'Date']

In [134]:
df = final2[['Line', 'Stat', 'Player', 'League', 'Team', 'Date']].copy(deep=True)
df

,Line,Stat,Player,League,Team,Date
0,22.5,points,Jalen Brunson,NBA,NYK,2023-10-31
1,30.5,pts_rebs_asts,Jalen Brunson,NBA,NYK,2023-10-31
2,5.0,assists,Jalen Brunson,NBA,NYK,2023-10-31
3,8.5,rebs_asts,Jalen Brunson,NBA,NYK,2023-10-31
4,25.5,pts_rebs,Jalen Brunson,NBA,NYK,2023-10-31
...,...,...,...,...,...,...
1566,16.5,fantasy_points,C. Seager + M. Semien,other,976becd7-ee88-4db5-9ae6-d1bb6b2b16a8,NaN
1567,16.5,fantasy_points,A. Garcia + N. Lowe,other,976becd7-ee88-4db5-9ae6-d1bb6b2b16a8,NaN
1568,13.5,fantasy_points,J. Jung + J. Heim,other,976becd7-ee88-4db5-9ae6-d1bb6b2b16a8,NaN
1569,14.5,fantasy_points,C. Walker + T. Pham,other,db69f844-d6eb-4332-b133-796ce316890a,NaN


In [135]:
df = df.reset_index(drop=True)

In [136]:
df.Stat.unique()

array(['points', 'pts_rebs_asts', 'assists', 'rebs_asts', 'pts_rebs',
       'pts_asts', 'steals', 'field_goals_att', 'three_points_att',
       'rebounds', 'personal_fouls', 'three_points_made', 'blocks',
       'turnovers', 'blks_stls', 'strikeouts', 'runs_allowed',
       'hits_allowed', 'fantasy_points', 'total_bases', 'hits_runs_rbis',
       'runs', 'singles', 'walks', 'batter_strikeouts', 'saves', 'shots',
       'blocked_shots', 'goals_against', 'goals', 'rushing_yds',
       'receiving_yds', 'kicking_points', 'field_goals_made',
       'passing_yds', 'receiving_rec', 'passing_and_rushing_yds',
       'rush_rec_tds', 'passing_tds', 'rush_rec_yds', 'shots_on_target',
       'fouls', 'aces', 'double_faults', 'sets_lost', 'breakpoints_won',
       'games_won', 'games_lost', 'games_played', 'kills_in_maps_1_2_3',
       'fantasy_points_on_map_1', 'fantasy_points_on_maps_1_2_3',
       'kills_in_map_1', 'fantasy_points_on_map_2', 'assists_in_map_2',
       'deaths_in_map_3', 'assist

In [137]:
syntax = {
    'pts_rebs_asts': 'PTS+TRB+AST',
    'pts_rebs': 'PTS+TRB',
    'points': 'PTS',
    'free_throws_made': 'FT',
    'fantasy_points': 'Fantasy',
    'rebounds': 'TRB',
    'assists': 'AST',
    'steals': 'STL',
    'blocks': 'BLK',
    'blks_stls': 'BLK+STL',
    'turnovers': 'TO',
    'rebs_asts': 'TRB+AST',
    'pts_asts': 'PTS+AST',
    'three_points_made': '3PM',
    'shots': 'Shots',
    'goals': 'Goals',
    'saves': 'Goalie Saves',
    'clearances': 'Clearances',
    'goals_and_assists': 'Goals and Assists',
    'tackles': 'Tackles',
    'interceptions': 'Interceptions',
    'crosses': 'Crosses',
    'goals_against': 'Goal Against',
    'game_1_and_2_kills': 'MAPS 1-2 Kills',
    'map_1_and_2_kills': 'MAPS 1-2 Kills',
    'Headshots on Maps 1+2': 'MAPS 1-2 Headshots',
    'strokes': 'Strokes',
    'bogeys_or_worse': 'Bogeys or Worse',
    'birdies_or_better': 'Birdies or Better',
    'aces': 'Aces',
    'breakpoints_won': 'Break Points Won',
    'games_won': 'Games Won',
    'games_lost': 'Games Lost',
    'total_games': 'Total Games',
    'double_faults': 'Double Faults',
    'service_games_lost': 'Service Games Lost',
    'Fastest Laps': 'Fastest Laps',
    'position': 'position',
    'passing_yds': 'Pass Yards',
    'rushing_yds': 'Rush Yards',
    'receiving_yds': 'Receiving Yards' ,
    'fantasy_points': 'Hitter Fantasy Score',
    'strikeouts': 'Pitcher Strikeouts',
    'hits_runs_rbis': 'PTS',
    'singles':'Singles',
    'total_bases': 'Total Bases',
    'walks': 'Walks Allowed',
    'hits': 'Hits Allowed',
                }

In [138]:
df = df.replace(syntax)

In [139]:
df.loc[df['League'] == 'ESPORTS']

,Line,Stat,Player,League,Team,Date
577,27.5,kills_in_maps_1_2_3,Xiaohu+Light,ESPORTS,NRG,2023-11-02
578,13.5,kills_in_maps_1_2_3,Theshy+Weiwei,ESPORTS,NRG,2023-11-02
579,23.8,fantasy_points_on_map_1,TheShy,ESPORTS,NRG,2023-11-02
580,61.3,fantasy_points_on_maps_1_2_3,TheShy,ESPORTS,NRG,2023-11-02
581,2.5,kills_in_map_1,TheShy,ESPORTS,NRG,2023-11-02
...,...,...,...,...,...,...
1538,36.5,kills_on_maps_1_2_3,Addicted,ESPORTS,SAW,2023-11-01
1539,38.5,kills_on_maps_1_2_3,Fragger,ESPORTS,SAW,2023-11-01
1540,46.5,kills_on_maps_1_2_3,brk,ESPORTS,SAW,2023-11-01
1541,38.0,kills_on_maps_1_2_3,k1zpawn,ESPORTS,SAW,2023-11-01


In [140]:
df.loc[df.Player=='Krimbo']

,Line,Stat,Player,League,Team,Date


In [141]:
today = dt.date.today()

In [142]:
str(today)

'2023-10-31'

In [143]:
df.columns = [x.lower() for x in df.columns]
df.to_csv(f'Lines/dog/doglines_{today.month}_{today.day}.csv')

In [144]:
df.loc[df.league=='ESPORTS']

,line,stat,player,league,team,date
577,27.5,kills_in_maps_1_2_3,Xiaohu+Light,ESPORTS,NRG,2023-11-02
578,13.5,kills_in_maps_1_2_3,Theshy+Weiwei,ESPORTS,NRG,2023-11-02
579,23.8,fantasy_points_on_map_1,TheShy,ESPORTS,NRG,2023-11-02
580,61.3,fantasy_points_on_maps_1_2_3,TheShy,ESPORTS,NRG,2023-11-02
581,2.5,kills_in_map_1,TheShy,ESPORTS,NRG,2023-11-02
...,...,...,...,...,...,...
1538,36.5,kills_on_maps_1_2_3,Addicted,ESPORTS,SAW,2023-11-01
1539,38.5,kills_on_maps_1_2_3,Fragger,ESPORTS,SAW,2023-11-01
1540,46.5,kills_on_maps_1_2_3,brk,ESPORTS,SAW,2023-11-01
1541,38.0,kills_on_maps_1_2_3,k1zpawn,ESPORTS,SAW,2023-11-01
